In [1]:
from __future__ import print_function

import os
import sys

import torch
print(torch.__version__)
import torch.backends.cudnn

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from bayes_opt import BayesianOptimization
from itertools import product

#import scripts
import importlib as importlib
sys.path.append(os.path.abspath('./helper_functions'))
import helper_functions.compute_stats as compute_stats
import parameters as params
import helper_functions.helper_functions as helper_functions
import helper_functions.neural_network as neural_network
import helper_functions.plot_data as plot_data
import helper_functions.load_data as load_data
import helper_functions.image_transform as image_transform
import helper_functions.generate_images as generate_images

#set gpu env
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device",device)
print("cuda version",torch.version.cuda)


2.3.0+cu121
2.3.0+cu121
2.3.0+cu121
2.3.0+cu121
2.3.0+cu121
Device cuda:0
cuda version 12.1
Device cuda:0
cuda version 12.1


In [2]:
params = params.Parameters()

# Prepare Data Pipeline

In [3]:
def generate_dataset_to_images_process(stock_ticker, params, test_size, cols_used):
    #import Financial Data
    stock_dataset_df = load_data.import_dataset(stock_ticker, params.start_date, params.end_date)

    # plot price comparison stock vs index
    plot_data.plot_price_comparison_stocks(params.index_ticker, stock_ticker, stock_dataset_df, params.start_date, params.end_date)

    # Generate images
    feature_image_dataset_list, feature_price_dataset_list, feature_label_dataset_list, cols_used_count = image_transform.generate_features_lists(
        stock_dataset_df, 
        cols_used,
        params.transform_algo, 
        params.transformed_img_sz, 
        params.gaf_method, 
        params.sample_range)

    images_array, labels_array = image_transform.create_images_array(feature_image_dataset_list, feature_label_dataset_list)

    #Quick Sample Image Visualization
    #Visualize Closing Price for one image in GAF or Markov:
    # A darker patch indicates lower correlation between the different elements of the price time series, 
    # possibly due to higher volatility or noise. The opposite is true for the lighter patches.
    if params.scenario == 0: plot_data.quick_view_images(images_array, cols_used_count, cols_used)

    #Prepare and Load Data
    images_array, labels_array = image_transform.squeeze_array(images_array, labels_array)

    feature_image_dataset_list_f32, labels_scaled_list_f32 = image_transform.Generate_feature_image_to_f32(
        labels_array, 
        images_array,
        params.transformed_img_sz, 
        params.scaler)

    train_loader, test_loader = load_data.Generate_Loaders(feature_image_dataset_list_f32,
                                                labels_scaled_list_f32, test_size,
                                                params.batch_size,
                                                train_shuffle=False)
    
    return train_loader, test_loader, stock_dataset_df

# Train Network Pipeline

In [4]:
def train_process(train_loader, params):
    #init cnn
    net = neural_network.instantiate_net(params)

    # train cnn
    net = neural_network.Train(params, train_loader, net)

    helper_functions.Save_Model(params.scenario, net)

    return net

# Test Network Pipeline

In [5]:
def test_process(net, test_loader, params, stock_ticker):
    
    # test
    stack_input, predicted_list, actual_list, accuracy, stack_actual, stack_predicted  = neural_network.Test(test_loader,net)

    # Plot image mean input values
    plot_data.scatter_diagram_onevar_plot_mean(stack_input, stock_ticker)
    
    #compute stats
    compute_stats.compute_and_report_error_stats(stack_actual, stack_predicted, stock_ticker)

    #write to file
    helper_functions.write_scenario_to_log_file(accuracy)

    return stack_input, stack_actual, stack_predicted

# External Validation Pipeline

In [6]:
# report stats results
def report_external_test_stats(params, stock_dataset_df, 
                               test_stack_input, train_stack_input,
                               test_stack_actual, test_stack_predicted):
    
    #test_stack_actual is the actual observation
    #test_stack_predicted is the predicted observation

    #print("test_stack_input.shape",test_stack_input.shape,"test_stack input",test_stack_input)
    #compute correl prices
    compute_stats.stock_correlation_matrix(params.external_test_stock_ticker, stock_dataset_df)
    #compute correl images
    #print("trained input image shape",train_stack_input.shape,"test input image shape",test_stack_input.shape)
    image_series_correlations, image_series_mean_correlation = compute_stats.cross_stock_image_array_correlation2(test_stack_input,train_stack_input)

    #compute cross correl
    benchmark_stock_df = load_data.import_dataset(params.train_stock_ticker, params.start_date, params.end_date)
    compute_stats.cross_stock_df_correlation(params.external_test_stock_ticker, params.train_stock_ticker,stock_dataset_df, benchmark_stock_df)

    #compute stats
    error_stats = compute_stats.compute_error_stats(test_stack_actual, test_stack_predicted)
    print(f"Inference Model Stats for {params.external_test_stock_ticker}")
    for key, value in error_stats.items():
        print(f'{key}: {value}')

    return image_series_correlations, image_series_mean_correlation

# <b>Brute Force Train and Test</b>

In [7]:
#################################
#       Train and Test          #
#################################
#instantiate params
#params = parameters.Parameters()

#generate training images
train_loader, test_loader, stock_dataset_df = generate_dataset_to_images_process(params.train_stock_ticker, 
                                                            params, 
                                                            params.training_test_size, 
                                                            params.training_cols_used)

net = train_process(train_loader, params)

#test
# set model to eval
net  = neural_network.set_model_for_eval(net)

test_stack_input, test_stack_actual, test_stack_predicted = test_process(net, test_loader, 
                                                                        params, 
                                                                        params.train_stock_ticker)

#################################
#       External Test           #
#################################
print ("\n\nRun External Stock Tests:")
#load model
PATH = f'./model_scen_{0}_full.pth'
net = helper_functions.Load_Full_Model(PATH)

#external test image generation
cols_used = ["Open", "High"]
train_loader, test_loader, stock_dataset_df = generate_dataset_to_images_process(params.external_test_stock_ticker, 
                                                            params, 
                                                            params.external_test_size, 
                                                            params.external_test_cols_used)

#test
external_test_stack_input, external_test_stack_actual, external_test_stack_predicted = test_process(net, 
                                                                                                    test_loader, 
                                                                                                    params,
                                                                                                    params.external_test_stock_ticker)

#report stats
image_series_correlations, image_series_mean_correlation = report_external_test_stats(
                                                    params, stock_dataset_df, 
                                                    test_stack_input, external_test_stack_input,
                                                    external_test_stack_actual, external_test_stack_predicted)

plot_data.plot_external_test_graphs(params, test_stack_input, external_test_stack_input,
                            image_series_correlations, image_series_mean_correlation)

AttributeError: module 'parameters' has no attribute 'train_stock_ticker'

-------------------------------------------------------

## Bayesian Optimization for CNN Hyperparameters

In [ ]:
def CNN_Optimization(output_conv_1, 
                     output_conv_2,
                     dropout_probab,
                     learning_rate,
                     momentum,
                     train_loader,test_loader,
                     iteration=None):
    
    output_conv_1 = int(output_conv_1)
    output_conv_2 = int(output_conv_2)
    
    net = neural_network.Net(output_conv_1=output_conv_1, output_conv_2=output_conv_2,dropout_probab=dropout_probab)
    net.to(device)

    neural_network.Train(learning_rate=learning_rate, train_loader=train_loader, net=net, momentum=momentum)

    if iteration is not None:
        helper_functions.Save_BayesOpt_Model(iteration,net)

    stack_input, predicted, actual, accuracy, percentage_diffs, mean_percentage_diff = neural_network.Test(test_loader=test_loader,net=net)
    
    print("accuracy received",accuracy)

    return accuracy

In [ ]:
def Optimize():
    
    train_loader,test_loader = load_data.Generate_Train_And_Test_Loaders(feature_image_dataset_list_f32,labels_scaled_list_f32, test_size, batch_size=batch_size, train_shuffle=False)
    
    iteration_counter = [0]

    def cnn_correct_pct_wrapper(output_conv_1, output_conv_2, learning_rate, dropout_probab, momentum):
        result = CNN_Optimization(output_conv_1, output_conv_2, learning_rate, dropout_probab, momentum, train_loader, test_loader, iteration=iteration_counter[0])
        iteration_counter[0] += 1
        return result
    #cnn_correct_pct = partial(CNN_Optimization, train_loader = train_loader, test_loader = test_loader)

    # Bounded region of parameter space
    pbounds = {'output_conv_1': (40, 80),
               'output_conv_2': (8, 16), 
               'learning_rate': (0.00001, 0.0001), 
               'dropout_probab': (0.0, 0.5), 
               'momentum': (0.8, 1.0)}

    optimizer = BayesianOptimization(
        f = cnn_correct_pct_wrapper,
        pbounds=pbounds,
        random_state=1,
        )

    #n_iter:steps of bayesian optimization you want to perform
    #init_points:steps of random exploration
    optimizer.maximize(init_points=10, n_iter=10,)

    with open(f'optimizer_results.txt', 'a') as file:
        for i, res in enumerate(optimizer.res):
            print(f"Iteration {i}: \n\t{res}")
            file.write("\n\nIteration {}: \n\t{}".format(i, res))

    optimizer_max_result = optimizer.max
    print("optimizer result:\n",optimizer.max)

    with open(f'optimizer_results.txt', 'a') as file:
        file.write('\noptimizer results:\n' + str(optimizer_max_result))

In [ ]:
# comment out to not run bayesian opt
#Optimize()

In [ ]:
# example to reconstruct Net model design. Enter optimizer_results.txt scenario variable results used to validate the net design model inference result matches the optimizer_results.txt output
# dropout_probab = 0.4473033317519236
# learning_rate = 1.7653979023280013e-05
# momentum = 0.8078109566465765
# output_conv_1 = int(46.7)
# output_conv_2 = int(15.0)

# net = Net(name='Classification Net', filter_size_1=filter_size_1, filter_size_2=filter_size_2,
#             filter_size_3=filter_size_3, stride=stride,
#             image_resolution_x=32,image_resolution_y=32,
#             output_conv_1=output_conv_1, output_conv_2=output_conv_2,
#             output_FC_1=output_FC_1, output_FC_2=output_FC_2,
#             final_FCLayer_outputs=final_FCLayer_output,
#             dropout_probab=dropout_probab)

# net.to(device)
# net.parameters()

# net = Load_BayesOpt_Model("5",net)

# predicted_np, actual_np, accuracy, percentage_diffs, mean_percentage_diff, average_percentage_diff, df = Test_And_Report(net)
